In [1]:
# Script criado pelo Prof. Eric Bastos Gorgens - UFVJM (eric.gorgens@ufvjm.edu.br) e Adaptado por Felipe Ribeiro
# Pequenas adaptações por Guilherme de Oliveira Leão - SEAPA (guilherme.leao@agricultura.mg.gov.br)
# Criação de imagem sentinel RGB + IR para bacia de interesse, e período de interesse
# Licensa uso Creative Commons - Atribuição-CompartilhaIgual 4.0 Internacional

# Neste estudo realizei um pré treinamento com algun locais com matas, cafés e não cafés, para que eu consiga com o EE
# realizar o estudo automaticamente
# O ID-1 é café, ID-2 é Mata e ID-3 é não café
# A catalogação do que é ou não café foi realizada com o uso e ocupação do solo de 2020 e com o Google Satélitie de
# Janiero de 2023, então os limiares das bandas devem ser próximo a este limiar de datas

In [1]:
# Importando arquivos
import ee
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [2]:
#Initializing earth engine
link_cred = "testestcc@reg-vaz.iam.gserviceaccount.com"
cred = ee.ServiceAccountCredentials(link_cred, r"Secrets/reg-vaz-9444ac120e33.json")
ee.Initialize(cred)

In [5]:
# Area de drenagem do ponto de estudo
# area = ee.FeatureCollection("Mapas/limite_mg.geojson")

# Definindo a imagem de exemplo para uma data específica
startDate = '2021-06-30'
endDate = '2021-07-20'

# Filtra imagens da coleção no período escolhido
# O Período pode ser alterado abaixo. Não esqueça de inserir um período que esteja compreendido
# no tempo mínimo de um estudo de Uso e Ocupação da Terra para o ZAP (6 meses anterior ao estudo)
# Formato: ANO-MÊS-DIA
sentinel:ee.Image = ee.ImageCollection("COPERNICUS/S2_SR").filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10).filterDate(startDate, endDate).median()

In [ ]:
# Determinando os valores do que é ou não café
# Abridno o arquivo com os poligonos datados
cafes:gpd.GeoDataFrame = gpd.read_file("Mapas/arquivos.gpkg", driver="GPKG", layer = "cafe")
trainingVals = []
index = 0
for row in cafes.itertuples():
	if index == 20:
		break

	geom = row.geometry.envelope
	ee_polygon = ee.Geometry.Polygon(list(geom.exterior.coords))
	ee_geometry = ee.Geometry(ee_polygon)

	mean = sentinel.clip(ee_geometry)

	mean = mean.reduceRegion(
		reducer=ee.Reducer.mean(),
		geometry=ee_geometry,
		scale=10,
		maxPixels=1e9
	)

	mean_info = mean.getInfo()

	mean_info['landcover'] = 1

	trainingVals.append(
		ee.Feature(
			ee_geometry,
			mean_info,
		),
	)

	print(row)

	index += 1

# EE collection for training
polygons = ee.FeatureCollection(trainingVals)

In [ ]:
# Testar todos e ver qual melhor resultado
# ee.Classifier.randomForest: Implementação de floresta aleatória, que é uma técnica de aprendizado de máquina baseada em múltiplos modelos de árvore de decisão.
# ee.Classifier.smileCart: Implementação da árvore de decisão de classificação e regressão (CART, na sigla em inglês).
# ee.Classifier.svm: Implementação de máquinas de vetor de suporte (SVM), que é uma técnica de aprendizado de máquina baseada em separação de hiperplanos.
# ee.Classifier.smileNaiveBayes: Implementação do algoritmo de Bayes ingênuo, que é uma técnica de classificação probabilística simples.
# ee.Classifier.gmo: Implementação do algoritmo de mínimos quadrados generalizados (GMO, na sigla em inglês), que é uma técnica de ajuste não-linear para modelos lineares generalizados.

trainedClassifier = ee.Classifier.smileRandomForest(10).train(polygons,'landcover',['B1', 'B2', 'B3', 'B4', 'B5', 'B7'])

classified = sentinel.classify(trainedClassifier)
print(classified.getInfo())

In [ ]:
# Recorta a imagem treinada na área de interesse
clipped:ee.Image = sentinel.clip(classified)

print(clipped.getDownloadURL(scale=10, crs='EPSG:4326'))